# Alexis Weintraub, RH

### This loads the sql extension:

In [1]:
%load_ext sql

## Q1: This creates the database:

In [2]:
%sql sqlite:///lab2.db3            --(Q1) Creates the database

0 rows affected.


[]

## Q2: The following creates the three tables from the instrucitons and inserts their data

### Q2.1: This creates a table for Sailors:

In [3]:
%%sql drop table if exists Sailors;     --drops the table if it already exists
create table Sailors(                   --(Q2.1) creates a table for Sailors
       sid       int primary key,       --sets sid as the primary key for the table
       sname     string,
       rating    int,
       age       float
);
insert into Sailors values('22', 'dustin', '7', '45.0');     --inserts data into the table
insert into Sailors values('31', 'lubber', '8', '55.5');
insert into Sailors values('58', 'rusty', '10', '35.0');
insert into Sailors values('10', 'redbeard', '9', '22.0');
insert into Sailors values('11', 'greenbeard', '12', '34.0');

 * sqlite:///lab2.db3
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

#### This prints the Sailors table:

In [4]:
%sql select * from Sailors;

 * sqlite:///lab2.db3
Done.


sid,sname,rating,age
22,dustin,7,45.0
31,lubber,8,55.5
58,rusty,10,35.0
10,redbeard,9,22.0
11,greenbeard,12,34.0


### Q2.2: This creates a table for Boats:

In [5]:
%%sql drop table if exists Boats;       --drops the table if it already exists
create table Boats(                     --(Q2.2) creates a table for Boats
       bid     int primary key,         --sets bid as the primary key for the table
       name    string,
       color   string
);
insert into Boats values('101', 'interlake', 'red');        --inserts data into the table
insert into Boats values('103', 'clipper', 'green');        --also adds data not in the instructions to run more accurate tests
insert into Boats values('102', 'boaty', 'red');
insert into Boats values('104', 'mayflower', 'yellow');

 * sqlite:///lab2.db3
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

#### This prints the Boats table:

In [6]:
%sql select * from Boats;

 * sqlite:///lab2.db3
Done.


bid,name,color
101,interlake,red
103,clipper,green
102,boaty,red
104,mayflower,yellow


### Q2.3: This creates a table for Reserves:

In [7]:
%%sql drop table if exists Reserves;              --drops the table if it already exists
pragma foreign_keys = ON;
create table Reserves(                            --(Q2.3) creates a table for Reserves
       sid     int,
       bid     int,
       day     date,
       primary key(sid, bid, day),                --makes sid, bid, and day primary key constraints
       foreign key(sid) references Sailors(sid),
       foreign key(bid) references Boats(bid)
);
insert into Reserves values('22', '101', '10/10/96');   --inserts data into the table
insert into Reserves values('58', '103', '11/12/96');   --also adds data not in the instructions to run more accurate tests
insert into Reserves values('58', '102', '11/14/96');
insert into Reserves values('31', '104', '10/26/96');
insert into Reserves values('31', '102', '10/25/96');
insert into Reserves values('10', '101', '11/25/96');
insert into Reserves values('11', '103', '12/25/96');
insert into Reserves values('11', '101', '12/24/96');

 * sqlite:///lab2.db3
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

#### This prints the Reserves table:

In [8]:
%sql select * from Reserves;

 * sqlite:///lab2.db3
Done.


sid,bid,day
22,101,10/10/96
58,103,11/12/96
58,102,11/14/96
31,104,10/26/96
31,102,10/25/96
10,101,11/25/96
11,103,12/25/96
11,101,12/24/96


## Q3: This finds names of Sailors who have reserved Boat 103
### Data is taken from the Sailors and the Reserves tables

In [9]:
%%sql
SELECT sname                          --Selects the sname column from the Sailors table
FROM   Sailors, Reserves              --Takes data from the Sailors and Reserves table
WHERE  Sailors.sid = Reserves.sid     --Sailors.sid and Reserves.sid must equal each other (these are the sailors that reserved boat 103)
AND bid = '103';                      --bid must equal 103

 * sqlite:///lab2.db3
Done.


sname
rusty
greenbeard


## Q4: This finds names of sailors who’ve reserved a red boat
### I completed this question in two different ways

#### The first way uses one SFW query with two AND conditons:

In [10]:
%%sql
SELECT DISTINCT sname                         --Selects the sname column from the Sailors table (uses DISTINCT in case some sailors reserved more than 1 red boat)
FROM Sailors, Boats, Reserves                 --Takes data from the Sailors, Boats, and Reserves table (this is inefficient)
WHERE Sailors.sid = Reserves.sid              --Sailors.sid and Reserves.sid must equal each other
AND Reserves.bid = Boats.bid                  --Reserves.bid and Boats.bid must also equal each other
AND color = 'red';                            --Color must also be red

 * sqlite:///lab2.db3
Done.


sname
dustin
rusty
lubber
redbeard
greenbeard


#### The second way uses a nested query and is more efficient:

In [11]:
%%sql
SELECT DISTINCT sname                         --Selects the sname column from the Sailors table (uses DISTINCT in case some sailors reserved more than 1 red boat)
FROM Sailors
WHERE sid IN(                                 --Filters the data so the code does not have to examine ALL the data in the 3 tables
            SELECT sid
            FROM Reserves, Boats
            WHERE Reserves.bid = Boats.bid
            AND Color = "red"
);

 * sqlite:///lab2.db3
Done.


sname
redbeard
greenbeard
dustin
lubber
rusty


## Q5: This finds names of sailors who have reserved a red or a green boat
### I also completed this question in two different ways

#### The first way uses one SFW query with two AND conditions and an OR condition:

In [12]:
%%sql
SELECT DISTINCT sname                 --Selects the sname column from the Sailors table
FROM Sailors, Boats, Reserves         --Takes data from the Sailors, Boats, and Reserves table
WHERE Sailors.sid = Reserves.sid      --Sailors.sid must equal each other
AND Reserves.bid = Boats.bid          --Reserves.bid and Boats.bid must also equal each other
AND color = 'red'                     --Color must be red OR green
OR color = 'green';

 * sqlite:///lab2.db3
Done.


sname
dustin
lubber
rusty
redbeard
greenbeard


#### The second way uses a nested query and is more efficient:

In [13]:
%%sql
SELECT DISTINCT sname                           --Selects the sname column from the Sailors table (uses DISTINCT in case some sailors reserved more than 1 red boat)
FROM Sailors
WHERE sid IN(                                   --Filters the data so the code does not have to examine ALL the data in the 3 tables
            SELECT sid
            FROM Reserves, Boats
            WHERE Reserves.bid = Boats.bid
            AND Color = 'red'
            OR Color = 'green'
);

 * sqlite:///lab2.db3
Done.


sname
redbeard
greenbeard
dustin
lubber
rusty


## Q6: This finds names of sailors who've reserved a red and a green boat
#### I use nested queries and an INTERSECT statement here, to find sailors who appear in the data for both red boat reservations AND green boat reservations

In [14]:
%%sql
SELECT DISTINCT sname                              --Selects the sname column from the Sailors table (uses DISTINCT in case some sailors reserved more than 1 red boat)
FROM Sailors
WHERE sid IN(                                      --Filters the data so the code does not have to examine ALL the data in the 3 tables
        SELECT sid
        FROM Reserves, Boats
        WHERE Reserves.bid = Boats.bid
        AND Color = "red"
    INTERSECT                                      --Looks at the intersection of reservations of red boats and reservations of green boats
        SELECT sid
        From Reserves, Boats
        WHERE Reserves.bid = Boats.bid
        AND Color = "green");

 * sqlite:///lab2.db3
Done.


sname
greenbeard
rusty


## Q7: Find names of sailors who’ve reserved only red boats
#### I use an EXCEPT statement here to eliminate the intersection of red and other colors, so what is returned is Sailors who have exclusively reserved red boats

In [15]:
%%sql
SELECT sname                                   --Selects the sname column from the Sailors table (uses DISTINCT in case some sailors reserved more than 1 red boat)
FROM Sailors
WHERE sid IN(                                  --Filters the data so the code does not have to examine ALL the data in the 3 tables
        SELECT sid
        FROM Reserves, Boats
        WHERE Reserves.bid = Boats.bid
        AND Color = "red"
    EXCEPT                                     --Uses EXCEPT to exclude sailors who reserved red boats AND boats of other colors
        SELECT sid
        From Reserves, Boats
        WHERE Reserves.bid = Boats.bid
        AND Color != "red");

 * sqlite:///lab2.db3
Done.


sname
redbeard
dustin
